<a href="https://colab.research.google.com/github/CoderAPS/HuBMAP/blob/main/HuBMAP_split_images_train_val_test_20230606.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
!ls /gdrive/MyDrive/DATA/HuBMAP/

annotation_info.csv  masks_4d		    test	   wsi_meta.csv
images_info.csv      polygons.jsonl	    tile_meta.csv
masks		     sample_submission.csv  train


In [4]:
# Import libraries
import pandas as pd
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt

In [5]:
annotation_info = pd.read_csv('/gdrive/MyDrive/DATA/HuBMAP/annotation_info.csv',index_col=0)

In [6]:
annotation_info.head()

,blood_vessel,glomerulus,unsure
0006ff2aa7cd,8,1,0
00168d1b7522,1,1,0
0033bbc76b6b,3,0,1
003504460b3a,7,0,0
004daf1cbe75,9,1,0


In [12]:
#@title hide
images_file_path = glob.glob('/gdrive/MyDrive/DATA/HuBMAP/train/*')

In [ ]:
#@title hide
#image_names = [image_path.split('/')[-1].split('.')[0] for image_path in images_file_path]

In [13]:
#@title hide
#images_info = pd.DataFrame({'id':image_names})
#images_info['has_mask'] = images_info.id.map(
#    lambda id: 'yes' if id in annotation_info.index.tolist() else 'no')
#images_info.head()

In [ ]:
#@title hide
#images_info.to_csv('/gdrive/MyDrive/DATA/HuBMAP/images_info.csv')

In [15]:
images_info = pd.read_csv('/gdrive/MyDrive/DATA/HuBMAP/images_info.csv',index_col=0)

In [16]:
images_info.head()

,id,has_mask
0,da693ca2093c,no
1,daf9eb07acd4,no
2,db047e105270,no
3,da5cca9855f5,yes
4,dbb355ee6c9a,no


In [17]:
# Split masks ids into blood vessel only,blood vessel and glomerulus etc..
bv_only = annotation_info[(annotation_info['blood_vessel'] > 0)&\
 (annotation_info['glomerulus'] == 0)&(annotation_info['unsure'] == 0)]

bv_gl = annotation_info[(annotation_info['blood_vessel'] > 0)&\
 (annotation_info['glomerulus'] > 0)&(annotation_info['unsure'] == 0)]

bv_gl_u = annotation_info[(annotation_info['blood_vessel'] > 0)&\
 (annotation_info['glomerulus'] > 0)&(annotation_info['unsure'] > 0)]

bv_u = annotation_info[(annotation_info['blood_vessel'] > 0)&\
 (annotation_info['glomerulus'] == 0)&(annotation_info['unsure'] > 0)]

u_only = annotation_info[(annotation_info['blood_vessel'] == 0)&\
 (annotation_info['glomerulus'] == 0)&(annotation_info['unsure'] > 0)]

In [18]:
# Print out the number of various mask according to grouping specified above
total = len(bv_only) + len(bv_gl) + len(bv_gl_u) + len(bv_u) + len(u_only)

print(f'1. Total numer of masks: {len(annotation_info)}')
print(f'2. Masks with blood vessel only: {len(bv_only)}')
print(f'3. Masks with blood vessel and glomerulus: {len(bv_gl)}')
print(f'4. Masks with blood vessel and unsure: {len(bv_u)}')
print(f'5. Masks with blood vessel, glomerulus and unsure: {len(bv_gl_u)}')
print(f'6. Masks with unsure: {len(u_only)}')
print(f'7. Sum of 2 to 6: {total}')

1. Total numer of masks: 1633
2. Masks with blood vessel only: 867
3. Masks with blood vessel and glomerulus: 407
4. Masks with blood vessel and unsure: 320
5. Masks with blood vessel, glomerulus and unsure: 28
6. Masks with unsure: 11
7. Sum of 2 to 6: 1633


In [15]:
#@title hide
#df = annotation_info[(~annotation_info.index.isin(bv_only.index))&(
#    ~annotation_info.index.isin(bv_gl.index))&(~annotation_info.index.isin(bv_u.index))&(
#        ~annotation_info.index.isin(bv_gl_u.index))]
#len(df)

11

In [ ]:
#@title hide
# Display masks with number of unsure per mask counts
u_only.unsure.value_counts().to_frame('counts').reset_index().rename(columns={'index':'num unsure objs'})

In [154]:
#@title hide
# Spliting of bv_only ids
# test -> 4
# val -> 78
# train -> 785

id_list = pd.Series(bv_only.index.tolist())
max_size = len(u_only) + len(bv_gl_u) + len(bv_u) + len(bv_gl) + len(bv_only)
common_list = []

test_size = len(test_list) + 4
val_size = len(val_list) + 78
train_size = len(train_list) + 785

while max_size > len(common_list) + len(u_only) + len(bv_gl_u) + len(bv_u) + len(bv_gl):
  id = id_list.sample(1).values[0]
  if len(test_list) < test_size and id not in common_list:
    test_list.append(id)
    common_list.append(id)
  elif len(val_list) < val_size and id not in common_list:
    val_list.append(id)
    common_list.append(id)
  elif len(train_list) < train_size and id not in common_list:
    train_list.append(id)
    common_list.append(id)

In [ ]:
#@title hide
print(f'Number of ids assigned to test_list: {len(test_list)}')
print(f'Number of ids assigned to val_list: {len(val_list)}')
print(f'Number of ids assigned to train_list: {len(train_list)}')

for idx,id in enumerate(bv_only.index.tolist()):
  if id not in test_list and id not in val_list and id not in train_list:
    print(f'{idx}:: {id}: is not in any of the three datasets!!!!')

In [20]:
# Split image/mask for each mask group into into test, val and train lists
# according to specified split 
def split_ids(data_sets,mask_group,max_size,splits,added_size):
  id_list = pd.Series(mask_group.index.tolist())
  common_list = []

  test_list, val_list, train_list = data_sets[0], data_sets[1], data_sets[2]
  test_size = len(test_list) + splits[0]
  val_size = len(val_list) + splits[1]
  train_size = len(train_list) + splits[2]

  while max_size > len(common_list) + added_size:
    id = id_list.sample(1).values[0]
    if len(test_list) < test_size and id not in common_list:
      test_list.append(id)
      common_list.append(id)
    elif len(val_list) < val_size and id not in common_list:
      val_list.append(id)
      common_list.append(id)
    elif len(train_list) < train_size and id not in common_list:
      train_list.append(id)
      common_list.append(id)
  
  return test_list, val_list, train_list


In [21]:
# Split mask ids into train, validation and test where the various masks groups 
# specified above are distributed anomg the three data set as specified
# total number of image/mask for the three datasets:
# train = 1467
# validation = 156
# test = 10

train_list = []
val_list = []
test_list = []

In [22]:
# Split mask group into the the three dataset list accoriding to the table below
dataset_split = pd.DataFrame({'u_only':[1,2,8,len(u_only)],
                              'bv_gl_u':[1,6,21,len(bv_gl_u)],
                              'bv_u':[2,30,288,len(bv_u)],
                              'bv_gl':[2,40,365,len(bv_gl)],
                              'bv_only':[4,78,785,len(bv_only)]},
                           index=['test','val','train','group_total'])

dataset_split['dataset_total'] = dataset_split.sum(axis=1,numeric_only=True)
dataset_split

,u_only,bv_gl_u,bv_u,bv_gl,bv_only,dataset_total
test,1,1,2,2,4,10
val,2,6,30,40,78,156
train,8,21,288,365,785,1467
group_total,11,28,320,407,867,1633


In [23]:
# Split mask group u_only
test_list,val_list,train_list = split_ids([test_list,val_list,train_list],
                                          u_only, len(u_only),[1,2,8],0)

# Split mask group bv_gl_u
test_list,val_list,train_list = split_ids([test_list,val_list,train_list],
                                          bv_gl_u, len(u_only)+len(bv_gl_u),
                                           [1,6,21],len(u_only))

# Split mask group bv_u
test_list,val_list,train_list = split_ids([test_list,val_list,train_list],
                                          bv_u, len(u_only)+len(bv_gl_u)+len(bv_u),
                                           [2,30,288],
                                          len(u_only)+len(bv_gl_u))

# Split mask group bv_gl
test_list,val_list,train_list = split_ids([test_list,val_list,train_list],
                                          bv_gl, 
                                          len(u_only)+len(bv_gl_u)+len(bv_u)+len(bv_gl),
                                           [2,40,365],
                                          len(u_only)+len(bv_gl_u)+len(bv_u))
# Split mask group bv_only
test_list,val_list,train_list = split_ids([test_list,val_list,train_list],
                                          bv_only, 
                                          len(u_only)+len(bv_gl_u)+len(bv_u)+len(bv_gl)+len(bv_only),
                                           [4,78,785],
                                          len(u_only)+len(bv_gl_u)+len(bv_u)+len(bv_gl))

In [24]:
print(f'Number of ids assigned to test_list: {len(test_list)}')
print(f'Number of ids assigned to val_list: {len(val_list)}')
print(f'Number of ids assigned to train_list: {len(train_list)}')

for i,id in enumerate(annotation_info.index.tolist()):
  if id not in test_list and id not in val_list and id not in train_list:
    print(f'{i}:: {id}: is not in any of the three datasets!!!!')

Number of ids assigned to test_list: 10
Number of ids assigned to val_list: 156
Number of ids assigned to train_list: 1467


In [26]:
test_list_ds = pd.Series(test_list) 
val_list_ds = pd.Series(val_list) 
train_list_ds = pd.Series(train_list) 

test_list_ds.to_csv('/gdrive/MyDrive/DATA/HuBMAP/test_list.csv')
val_list_ds.to_csv('/gdrive/MyDrive/DATA/HuBMAP/val_list.csv')
train_list_ds.to_csv('/gdrive/MyDrive/DATA/HuBMAP/train_list.csv')

In [27]:
!ls /gdrive/MyDrive/DATA/HuBMAP

annotation_info.csv  polygons.jsonl	    tile_meta.csv   wsi_meta.csv
images_info.csv      sample_submission.csv  train
masks		     test		    train_list.csv
masks_4d	     test_list.csv	    val_list.csv


In [ ]:
img_height = 512
img_width =512
channels = 3

train_imges = np.zeros((1470,img_height,img_width,channels),dtype=np.uint8)
val_imges = np.zeros((153,img_height,img_width,channels),dtype=np.uint8)
test_iamges = np.zeros((10,img_height,img_width,channels),dtype=np.uint8)